# Aritmética modular

## Ejercicio 1

Implementa el algoritmo extendido de Euclides para el cálculo del máximo común divisor: dados dos enteros $a$ y $b$, encuentra $u, v ∈ \mathbb{Z}$ tales que $au + bv$ es el máximo común divisor de $a$ y $b$.

In [1]:
ext_euclides :: Integral a => a -> a -> [a]
ext_euclides a b = ext_euclides' a b

ext_euclides' :: Integral a => a -> a -> [a]
ext_euclides' a 0 = [a, 1, 0]
ext_euclides' 0 b = [b, 0, 1]
ext_euclides' a b = [d, m, n - (a `div` b) * m]
    where
        [d,n,m] = ext_euclides' b (a `mod` b)

ext_euclides 4864 3458

[38,32,-45]

En el código podemos ver como la función `ext_euclides` recibe como parámetros de entrada dos enteros $a$ y $b$ y devuelve el máximo común divisor, seguidos por $u$ y $v$. 

La función sigue el ejemplo de código del algoritmo _2.107_ de [A. Menezes, P. van Oorschot, and S. Vanstone, Handbook of Applied Cryptography, CRC Press, 1996.](http://cacr.uwaterloo.ca/hac/about/chap2.pdf)

## Ejercicio 2

Usando el ejercicio anterior, escribe una función que calcule $a^{-1} \bmod b$ para cualesquiera $a, b$ enteros que sean primos relativos.

In [2]:
inverse :: Integral a => a -> a -> a
inverse a b = ext_euclides a b !! 1 `mod` b

inverse 2 5

3

A partir del código del ejercicio 1, en caso de que exista inversa en $\mathbb{Z}_n$, obtendremos lo siguiente: $$d = au + bv$$ En caso de que $a$ tenga inversa en $\mathbb{Z}_n$, tendremos que $\text{mcd}(a,n) = 1$. Por tanto, por la identidad de Bezout, tenemos que existen $u$ y $v$ (coeficientes de Bezout) tal que: $$1 = ua + vn$$

Por tanto, si estamos en el espacio $\mathbb{Z}_n$, tenemos que $$ \begin{matrix}1 = ua + vn & =& ua + 0 \\ & \Rightarrow & ua & a \in \mathcal{U}(\mathbb{Z}_n)\\  u & = & a^{-1} \end{matrix}$$

Para devolver el inverso correcto, devolveremos $u \bmod n$.

## Ejercicio 3

Escribe una función que calcule $a^b \bmod n$ para cualesquiera $a, b\text{ y } n$. La implementación debe tener en cuenta la representación binaria de $b$.

In [3]:
big_pow :: Integral a => a -> a -> a -> a
big_pow _ 0 _ = 1
big_pow a b n = pow a b 1 n

pow :: Integral a => a -> a -> a -> a -> a
pow _ 0 p _ = p
pow a b p n 
        | b `mod` 2 == 1 = pow (a*a `mod` n) (b `div` 2) ((p * a) `mod` n) n
        | otherwise      = pow (a*a `mod` n) (b `div` 2) p n

        
big_pow 5 35 7

3

Para calcular $a^b \bmod n$, podemos tomar como base que, el exponente $b$ puede escribirse en binario como $b = b_0b_1\ldots b_k$ tal que $b_i = 0\;|\;1$. A partir de esto, podemos definir $b$ como $b = \sum_{i=0}^k b_i\cdot2^i$.

Entonces, la expresión $a^b$ se puede expresar como: $$a^b = a^{\sum_{i=0}^k b_i\cdot2^i} = \prod a^{b_i2^i} = \prod \left(a ^{2^i}\right)^{b_i}$$

Con esto podemos ver que el valor del producto se incrementará cuando el valor de $b_i = 1$, elevándose el valor del producto al cuadrado.

Una forma de realizar este cálculo, es la que aparece en el código, y es ir realizando las operaciones $b_i = b \bmod 2; \quad b = \lfloor b | 2 \rfloor$, e ir siempre incrementando el valor de $a$ como $a = a^2 \bmod n$. El valor del producto, denotado como $p$, se incrementará cuando el valor de $b = 1$, tal que $p = (p \cdot a)\bmod n$.

## Ejercicio 4

Dado un entero $p$, escribe una función para determinar si $p$ es probablemente primo usando el método de Miller-Rabin.

In [4]:
import System.Random
import System.IO.Unsafe
import Data.List

bifactor :: Integral a => a -> [a]
bifactor num = bifactor' num 0

bifactor' :: Integral a => a -> a -> [a]
bifactor' 0 s = [s, 1]
bifactor' a0 s 
    | a0 `mod` 2 == 0 = bifactor' (a0 `div` 2) (s + 1)
    | otherwise       = [s, a0]


miller_rabin :: (Integral a, Random a) => a -> Bool
miller_rabin p = test_mr p l
    where
        s_u = bifactor (p - 1)
        a = unsafePerformIO $ randomRIO (2, p - 2)
        l = map (\x -> big_pow a ((2^x)*s_u !! 1) p) [0..s_u !! 0]

        test_mr :: (Integral a) => a -> [a] -> Bool
        test_mr p l
            | (head l) == 1 || (head l) == (p - 1) = True
            | (p - 1) `elem` l && (last l /= p - 1) = True
            | not (1 `elem` l) = False
            | 1 `elem` l && (last $ takeWhile (/= 1) l) /= (p - 1) = False
            | otherwise = False


miller_rabin_test :: (Integral a, Random a) => a -> Int -> Bool
miller_rabin_test p n = and $ replicate n (miller_rabin p)

miller_rabin_test 123456789101119 10

True

El test de Miller-Rabin lo realiza la función `miller_rabin`. Esta función se encarga de dado un "primo" $p$, se encarga de descomponer $p-1$ en $p-1 = 2^s\cdot u$. 

Con esto compone la lista $l = [a^{2^0u}, a^{2^0u}, \ldots, a^{2^su}]$, siendo $a \in_R [2, \ldots, p - 2]$ y pasamos a comprobar las condiciones de ser ___probablemente primo___ o ___no primo___:
* Si el primer elemento de $l$ es 1 o -1, $p$ será probablemente primo.
* Si no aparece 1 en $l$, es no primo, ya que no supera el test de Fermat.
* Si aparece un 1 no precedido de -1, es no primo, ya que existe un raíz cuadrada de 1 que no es ni 1 ni -1.
* Si aparece un -1 en $l$, es probablemente primo, ya que el siguiente elemento en la lista será 1.

## Ejercicio 5

Implementa el algoritmo paso enano-paso gigante para el cálculo de algoritmos discretos en $\mathbb{Z}_p$.